In [1]:
import nltk
from nltk.stem.porter import *
from torch.nn import *
from torch.optim import *
import numpy as np
import pandas as pd
import torch,torchvision
import random
from tqdm import *
from torch.utils.data import Dataset,DataLoader
stemmer = PorterStemmer()

In [2]:
def tokenize(sentence):
    return nltk.word_tokenize(sentence)

In [3]:
tokenize('$test')

['$', 'test']

In [4]:
def stem(word):
    return stemmer.stem(word.lower())

In [5]:
stem('organic')

'organ'

In [6]:
def bag_of_words(tokenized_words,all_words):
    tokenized_words = [stem(w) for w in tokenized_words]
    bag = np.zeros(len(all_words),dtype=np.float32)
    for idx,w in enumerate(all_words):
        if w in tokenized_words:
            bag[idx] = 1.0
    return bag

In [7]:
bag_of_words(['hi','how','are','you','?','1000','$'],['$','money','1000 dollors','you','gregre','getgwe','fgrfg'])

array([1., 0., 0., 1., 0., 0., 0.], dtype=float32)

In [8]:
data = pd.read_csv('./data.csv').dropna()[:12500]

In [9]:
data

,Unnamed: 0,blurb,state
0,1,"Using their own character, users go on educati...",failed
1,2,"MicroFly is a quadcopter packed with WiFi, 6 s...",successful
2,3,"A small indie press, run as a collective for a...",failed
3,4,Zylor is a new baby cosplayer! Back this kicks...,failed
4,5,Hatoful Boyfriend meet Skeletons! A comedy Dat...,failed
...,...,...,...
12495,12496,A film about a place in Afghanistan where peop...,successful
12496,12497,"""The Great Controversy Ended"" Movie/Doc based ...",successful
12497,12498,A movie about filmmaker Alex Sichel and her fi...,successful
12498,12499,Documentary and multimedia project about women...,successful


In [10]:
X = data['blurb']

In [11]:
y = data['state']

In [12]:
all_words = []
all_data = []
tags = []

In [13]:
for X_batch,y_batch in tqdm(zip(X,y)):
    X_batch = tokenize(X_batch)
    new_X = []
    for X_batch_in_X_batch in X_batch:
        new_X.append(stem(X_batch_in_X_batch))
    all_words.extend(new_X)
    all_data.append((new_X,y_batch))
    tags.append(y_batch)

12500it [00:04, 2649.96it/s]


In [14]:
np.random.shuffle(all_words)
np.random.shuffle(all_data)
np.random.shuffle(tags)

In [15]:
all_words = sorted(set(all_words))

In [16]:
tags = sorted(set(tags))

In [17]:
tags

['failed', 'successful']

In [18]:
np.random.shuffle(all_words)
np.random.shuffle(all_data)
np.random.shuffle(tags)

In [19]:
tags

['successful', 'failed']

In [20]:
X = []
y = []

In [ ]:
for sentence,tag in tqdm(all_data):
    X.append(bag_of_words(sentence,all_words))
    y.append(tags.index(tag))

 96%|██████████████████████████████████▌ | 12007/12500 [00:43<00:01, 264.91it/s]

In [ ]:
from sklearn.model_selection import *

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.25,shuffle=False)

In [ ]:
device = 'cuda'

In [ ]:
X_train = torch.from_numpy(np.array(X_train)).to(device).float()
y_train = torch.from_numpy(np.array(y_train)).to(device).to(device).float()
X_test = torch.from_numpy(np.array(X_test)).to(device).to(device).float()
y_test = torch.from_numpy(np.array(y_test)).to(device).to(device).float()

In [ ]:
def get_loss(model,X,y,criterion):
    preds = model(X)
    loss = criterion(preds.view(-1,1),y.view(-1,1))
    return loss.item()

In [ ]:
def get_accuracy(model,X,y):
    correct = 0
    total = 0
    preds = model(X)
    for pred,y_batch in zip(preds,y):
        pred = int(torch.round(pred))
        y_batch = int(torch.round(y_batch))
        if pred == y_batch:
            correct += 1
        total += 1
    acc = round(correct/total,3)*100
    return acc

In [ ]:
class Model(Module):
    def __init__(self):
        super().__init__()
        self.activation = ReLU()
        self.hidden = 256
        self.iters = 10
        self.linear1 = Linear1(len(all_words),self.hiddendden)
        self.linear2 = Linear1(self.hidden,self.hidden)
        self.batchnorm = BatchNorm1d(self.hidden)
        self.output = Linear(self.hidden,1)
        self.output_ac = Sigmoid()
    def forward(self,X):
        preds = self.linear1(X)
        for _ in range(self.iters):
            preds = self.batchnorm(self.linear2(preds))
        preds = self.output_ac(self.output(preds))
        return preds

In [ ]:
model = Model().to(device)

In [ ]:
criterion = MSELoss()

In [ ]:
optimizer = Adam(model.parameters(),lr=0.001)

In [ ]:
import wandb

In [ ]:
PROJECT_NAME = 'kickstarter-NLP'

In [ ]:
epochs = 1000

In [ ]:
batch_size = 32

In [ ]:
wandb.init(project=PROJECT_NAME,name='baseline')
wandb.watch(model)
for _ in tqdm(range(epochs)):
    torch.cuda.empty_cache()
    for idx in range(0,len(X_train),batch_size):
        torch.cuda.empty_cache()
        X_batch = X_train[idx:idx+batch_size].to(device)
        y_batch = y_train[idx:idx+batch_size].to(device)
        preds = model(X_batch)
        loss = criterion(preds,y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    wandb.log({'Loss':get_loss(model,X_train,y_train,criterion)})
    wandb.log({'Val Loss':get_loss(model,X_test,y_test,criterion)})
    wandb.log({'Acc':get_accuracy(model,X_train,y_train)})
    wandb.log({'Val Acc':get_accuracy(model,X_test,y_test)})
wandb.watch(model)
wandb.finish()